In [1]:
import xarray as xr
import psutil
import os
from dask.distributed import Client

## Checking total memory with psutil

In [2]:
# Get total memory in bytes and convert to GB
total_memory = psutil.virtual_memory().total / (1024**3)
print(f"Total memory: {total_memory:.2f} GB")

Total memory: 63.25 GB


## Chunk Calculator

In [3]:
# Dimensions and data type
time_chunk = 365
latitude_chunk = 281
longitude_chunk = 441
data_size = 4  # float32 takes 4 bytes

# Calculate chunk size in bytes
chunk_size = time_chunk * latitude_chunk * longitude_chunk * data_size
chunk_size_GB = chunk_size / (1024**3)  # Convert to GB
print(f"Each chunk is approximately {chunk_size_GB:.6f} GB")

Each chunk is approximately 0.168499 GB


In [4]:
# Assuming you know your total memory (e.g., 16GB)
memory_limit = '8GB'  # Limit memory usage to 8GB per worker
client = Client(n_workers=4,memory_limit=memory_limit)  # Starts a local cluster with memory limits
print(client)

<Client: 'tcp://127.0.0.1:58561' processes=4 threads=24, memory=29.80 GiB>


In [5]:
client

<Client: 'tcp://127.0.0.1:64325' processes=4 threads=24, memory=29.80 GiB>

In [ ]:
# Open the NetCDF file
fname = "era5_t2min_1970_2000.nc"
ds = xr.open_dataset("data/{}".format(fname),chunks={"latitude": "auto", "longitude": 25,"time": -1}) #can set up chunks, for example xr.open_dataset("data/{}".format(fname),chunks={'time': 365})
#if more than one file, then use xr.open_mfdataset

In [8]:
ds

<xarray.Dataset> Size: 1GB
Dimensions:       (time: 1460, latitude: 281, longitude: 441)
Coordinates:
  * latitude      (latitude) float32 1kB 90.0 89.75 89.5 ... 20.5 20.25 20.0
  * longitude     (longitude) float32 2kB -150.0 -149.8 -149.5 ... -40.25 -40.0
  * time          (time) datetime64[ns] 12kB 1950-01-01 ... 2014-12-31
Data variables:
    daily_t2_max  (time, latitude, longitude) float32 724MB dask.array<chunksize=(365, 281, 441), meta=np.ndarray>
    daily_t2_min  (time, latitude, longitude) float32 724MB dask.array<chunksize=(365, 281, 441), meta=np.ndarray>

## Calculate seasonal anomalies

In [ ]:
da = ds["daily_t2_min"]
season_mean = da.groupby("time.season").mean(dim='time')
season_stdev = da.groupby("time.season").std(dim='time')
season_mean = season_mean.sel(season='DJF').drop_vars('season')
season_stdev = season_stdev.sel(season='DJF').drop_vars('season')

In [9]:
ds['anomaly'] = (da - season_mean)
ds['anomaly_scaled'] = ds['anomaly']/season_stdev

In [10]:
ds

<xarray.Dataset> Size: 46MB
Dimensions:         (latitude: 281, longitude: 441, time: 31)
Coordinates:
  * latitude        (latitude) float32 1kB 90.0 89.75 89.5 ... 20.5 20.25 20.0
  * longitude       (longitude) float32 2kB -150.0 -149.8 ... -40.25 -40.0
  * time            (time) datetime64[ns] 248B 1970-02-07 ... 2000-02-07
Data variables:
    daily_t2_min    (time, latitude, longitude) float32 15MB dask.array<chunksize=(31, 281, 25), meta=np.ndarray>
    anomaly         (time, latitude, longitude) float32 15MB dask.array<chunksize=(31, 281, 25), meta=np.ndarray>
    anomaly_scaled  (time, latitude, longitude) float32 15MB dask.array<chunksize=(31, 281, 25), meta=np.ndarray>

In [11]:
ds.to_netcdf(os.path.join("temp/", 'era5_t2min_scaled_anomalies.nc'),compute=True,mode="w")

In [9]:
#Remember to close the dask client
client.close()